In [1]:
import cv2
import numpy as np
import copy
import time
import os.path as osp
import os
import rospy
from cv_bridge import CvBridge
from std_msgs.msg import Header
from sensor_msgs.msg import PointCloud2 as Pc2
import sensor_msgs.point_cloud2 as p_c2
from sensor_msgs.msg import PointField as Pf
from sensor_msgs.msg import Image
import pdb
import sys
sys.path.append('../src')
from config import cfg

# Load data
# Input: a pointcloud numpy file from julian labeling (name_labeled or name_labeled_spaces)
# path = osp.join(cfg.DATASETS_DIR,'fsr_icp')

path = osp.join(cfg.ROOT_DIR,'lidar_classifier/logs/fsr_logs/icp_pred/julian2')

filename = [
    '10_pred',
    '13_pred',
    '15_pred',
    '16_pred',
    '18_pred',
    '1_pred',
    '5_pred',
    '6_pred',
]   

for f in filename:
    
    print("Converting file %s..." % f)
    
    pcl = np.load(osp.join(path,f+'.npy'))

    pcl_new = []
    for p in pcl:
        
        p_diff = p[np.sum(p[:,:3]-p[:,5:8],axis=1) !=0 ,:] # Find echo1 points that differ from echo2
        labels_echo1 = p_diff[:,-3:]
        labels_echo1 = np.sum(labels_echo1 * np.array([0,1,2]),axis=1)
        p_echo1 = np.concatenate((p_diff[:,:4],1*np.ones((p_diff.shape[0],1)),labels_echo1.reshape(-1,1)),axis=1)
        
        labels_echo2 = np.zeros((p_diff.shape[0],1))
        p_echo2 = np.concatenate((p_diff[:,5:9], 2*np.ones((p_diff.shape[0],1)),labels_echo2),axis=1) # Add 2 for second echo and label=0 because assumed non-particle
        
        p_both = p[np.sum(p[:,:3]-p[:,5:8],axis=1) == 0,:]
        labels_both = p_both[:,-3:]
        labels_both = np.sum(labels_both * np.array([0,1,2]),axis=1)
        p_both = np.concatenate((p_both[:,:4],np.zeros((p_both.shape[0],1)),labels_both.reshape(-1,1)),axis=1)
        pcl_new.append(np.concatenate((p_both,p_echo1,p_echo2)))
    
    np.save(osp.join(path,f+'_converted.npy'),pcl_new)
print('Conversion finished')

Converting file 10_pred...
Converting file 13_pred...
Converting file 15_pred...
Converting file 16_pred...
Converting file 18_pred...
Converting file 1_pred...
Converting file 5_pred...
Converting file 6_pred...
Conversion finished
